In [2]:
import configparser
import reads3
import sparkinit
import transform

# Load config file
config = configparser.ConfigParser()
config.read('dl.cfg')

['dl.cfg']

In [3]:
# Create spark session
ss = sparkinit.create_spark_session(config)

In [4]:
# Read log data from s3
log_path = "s3a://udacity-dend/log_data/2018/11/2018-11-0*.json"
staging_events = reads3.read_logs(ss, log_path)
staging_events.count()

1507

In [5]:
# Read song data from S3
song_path = "s3a://udacity-dend/song_data/A/A/*/*.json"
staging_songs = reads3.read_songs(ss, song_path)
staging_songs.count()

604

In [6]:
# Transform the dimension tables
songs = transform.select_songs_data(staging_songs)
artists = transform.select_artists_data(staging_songs)
users = transform.select_users_data(staging_events)
time = transform.select_time_data(staging_events)

In [7]:
# Create the songplay fact table
## Cache the songs and artists table in order not to recalculate it
songs = songs.cache()
artists = artists.cache()
songplay = transform.create_songplay(staging_events, songs, artists)

In [ ]:
# Write the data to S3
output_path = "s3a://dendpaulogieruswest2/p4csv/"
songs.write.partitionBy("year", "artist_id").parquet(output_path + 'songs/')
artists.write.parquet(output_path + 'artists/')
users.write.parquet(output_path + 'users/')
time.write.partitionBy("year", "month").parquet(output_path + 'time/')
songplay.write.partitionBy("year", "month").parquet(output_path + 'songplay/')